In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/MP2_FaceMask_Dataset.zip
!unzip -qq MP2_FaceMask_Dataset.zip
print("Data Downloaded Successfuly!!")

In [ ]:
!unzip -qq train.zip

In [ ]:
import numpy as np
import PIL
from matplotlib import pyplot as plt
import glob, os
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
import os, glob
from PIL import Image
import pandas as pd

In [ ]:
TRAINING_DIR = "/content/MP2_FaceMask_Dataset/train/"
VALIDATION_DIR = "/content/MP2_FaceMask_Dataset/test/"
IMG_SIZE = 64

In [ ]:
categories=os.listdir(TRAINING_DIR)
print('Categories Name :' ,categories)
labels=[i for i in range(len(categories))]
print('After convert it to 0,1 :',labels)

label_dict=dict(zip(categories,labels))
print(label_dict)
print('len of Labels dictionary : ' , len(label_dict))
# Preparing test data
categories_tst=os.listdir(VALIDATION_DIR)
print('Categories Name :' ,categories)
labels_tst=[i for i in range(len(categories))]
print('After convert it to 0,1 :',labels_tst)
label_dict_tst=dict(zip(categories,labels))
print(label_dict_tst)
print('len of Labels dictionary : ' , len(label_dict_tst))

In [ ]:
import cv2
Imges_Data_tst=[]
target_tst=[]


for  i in categories :
    path_Cat_tst=os.path.join(VALIDATION_DIR,i)
    print('Path Now is :', path_Cat_tst)
    Img_Names=os.listdir(path_Cat_tst)
    print(Img_Names[0:10])
    
    for img in Img_Names :
        img_path=os.path.join(path_Cat_tst,img)
        imge=cv2.imread(img_path)
   
        gray_img=cv2.cvtColor(imge,cv2.COLOR_BGR2GRAY)           

        dim=(IMG_SIZE,IMG_SIZE)
        resized_img = cv2.resize(gray_img, dim)
        Imges_Data_tst.append(resized_img)

        target_tst.append(label_dict[i])




In [ ]:
import cv2
Imges_Data_Train=[]
target_Train=[]


for  i in categories :
    path_Cat=os.path.join(TRAINING_DIR,i)
    print('Path Now is :', path_Cat)
    Img_Names=os.listdir(path_Cat)
    print(Img_Names[0:10])
    
    for img in Img_Names :
        img_path=os.path.join(path_Cat,img)
        imge=cv2.imread(img_path)
   
        gray_img=cv2.cvtColor(imge,cv2.COLOR_BGR2GRAY)           

        dim=(IMG_SIZE,IMG_SIZE)
        resized_img = cv2.resize(gray_img, dim)
        Imges_Data_Train.append(resized_img)

        target_Train.append(label_dict[i])

In [ ]:

Imges_Data_Train=np.array(Imges_Data_Train)/255.0
Imges_Data_Train=np.reshape(Imges_Data_Train,(Imges_Data_Train.shape[0],IMG_SIZE,IMG_SIZE,1))
Imges_Data_tst=np.array(Imges_Data_tst)/255.0
Imges_Data_tst=np.reshape(Imges_Data_tst,(Imges_Data_tst.shape[0],IMG_SIZE,IMG_SIZE,1))

In [ ]:
noise_factor = 0.4
x_train_noisy = Imges_Data_Train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=Imges_Data_Train.shape) 
x_test_noisy = Imges_Data_tst + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=Imges_Data_tst.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

#Displaying images with noise
plt.figure(figsize=(20, 2))
for i in range(1,10):
    ax = plt.subplot(1, 10, i)
    plt.imshow(x_test_noisy[i].reshape(64, 64), cmap="binary")
plt.show()

In [ ]:
noise_factor = 0.2
x_train_noisy = Imges_Data_Train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=Imges_Data_Train.shape) 
x_test_noisy = Imges_Data_tst + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=Imges_Data_tst.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

#Displaying images with noise
plt.figure(figsize=(20, 2))
for i in range(1,10):
    ax = plt.subplot(1, 10, i)
    plt.imshow(x_test_noisy[i].reshape(64, 64), cmap="binary")
plt.show()

In [ ]:
noise_factor = 0.05
x_train_noisy = Imges_Data_Train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=Imges_Data_Train.shape) 
x_test_noisy = Imges_Data_tst + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=Imges_Data_tst.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

#Displaying images with noise
plt.figure(figsize=(20, 2))
for i in range(1,10):
    ax = plt.subplot(1, 10, i)
    plt.imshow(x_test_noisy[i].reshape(64, 64), cmap="binary")
plt.show()

In [ ]:
import tensorflow as tf
tensor1 = tf.convert_to_tensor(x_train_noisy,dtype=tf.float64)
tensor2 = tf.convert_to_tensor(Imges_Data_Train,dtype=tf.float64)
tensor3 = tf.convert_to_tensor(x_test_noisy,dtype=tf.float64)
tensor4 = tf.convert_to_tensor(Imges_Data_tst,dtype=tf.float64)


# Model Paramet
Batch_size = 32

# Instantiate ImageDataGenerator
gen = ImageDataGenerator(rescale = 1.0/255, 
                         width_shift_range = 0.005,
                         height_shift_range = 0.005,
                         rotation_range = 0,
                         horizontal_flip = True)

# Generate batches of tensor image data
train_batches = gen.flow(tensor1, tensor2, batch_size = batch_size)
#Imges_Data_tst.shape,target_tst.tst
val_batches = gen.flow(tensor3, tensor4, batch_size = batch_size)

In [ ]:
train_batches

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
model = Sequential()
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(120, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
 

model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(120, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(1, (3, 3), activation='relu', padding='same'))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])
model.summary()

In [ ]:
tensor1.shape

In [ ]:
model.fit(train_batches, epochs=500, batch_size= Batch_size, shuffle=True, validation_data=(x_test_noisy, Imges_Data_tst))

In [ ]:
model.evaluate(x_test_noisy, x_test)

In [ ]:
model.save('denoising_autoencoder.model')

In [ ]:
plt.figure(figsize=(40, 4))
for i in range(10):
  ax = plt.subplot(3, 20, i + 1)
  plt.imshow(x_test_noisy[i].reshape(28, 28), cmap="binary")
    
  ax = plt.subplot(3, 20, 40 +i+ 1)
  plt.imshow(Imges_Data_tst[i].reshape(28, 28), cmap="binary")
  
plt.show()